# Part 3 - History Dependence

In the previous session, we implicitely made the assumption that the our neuron's spiking behaviour depends only on time: We created a model whose components (basis functions) were functions of the step number, but not of anything else. We might justify this by saying that the neuron's behaviour is mostly determined by some external stimulus, and that our model is basically an attempt to recover the dynamics of that stimulus.

But neurons also have hidden, intrinsic processes. Take the refractory period, for instance: Directly after each spike, it is either impossible to fire another spike, or, a bit later, it would require a stronger external signal than in case there had not been a spike recently. So in other words, the spike probability changes depending on whether there was a spike recently (for example dropping to zero after each spike, then slowly recover).
Furthermore, neurons interact, so one neuron's spike probability can be modulated by the spikes of *other* neurons that occured in previous steps.

In this part, we will therefore expand our generalized linear model to include history features. That is, besides the purely time-dependent spline basis functions, its design matrix will include columns that in some way reflect the spiking history of the current cell (auto-history) as well as that of other cells (cross-history). 

The motivation for this is twofold: First, we hope the additional information will lead to more accurate models, so that we can classify with higher accuracy from which cell or which condition a given spike train comes (hint, hint). Second, the new model consists of separate "components" that we can interpret: We'll see what influence a spike on our own neuron or some other neuron has on the firing probability, and we can now take the remaining, purely time-dependent model component as a more accurate reflection of the external stimulus, because that component now doesn't need to account for weird neuron-internal effects anymore. So in that sense, the decoding will get more informative.

**First, execute these code snippets from part 2 (data loading, resolution reduction, adaptive knot placement...):**

In [1]:
from matplotlib.pyplot import *
%matplotlib inline
import numpy as np
import splines
from scipy.ndimage.filters import gaussian_filter1d

def reduce_resolution(array,factor,sum_spikes=True):
    """ 
    Reduces the temporal resolution of an array of spike trains.
    
    This can cause multiple spikes to fall into one time bin, i.e. the
    returned array can contain integers > 1.
    If sum_spikes is set to False, we do not sum spike counts. Instead,
    any spike counts > 1 in one bin are truncated down to 1. In that case,
    a binary array is returned, but some spikes may be lost.
    
    Arguments:
        array: 4D numpy array. Reduction will occur along the last dimension (T).
        factor: integer to divide the number of time bins by.
        sum_spikes: If false, truncate the number of spikes per bin to 1.
      
    Returns:
        4D numpy array, similar as input but last dimension with length T/factor.
    """
    assert type(factor) is int
    reduced_shape = (array.shape[0],array.shape[1],array.shape[2],array.shape[3]/np.round(factor))
    spikes_reduced = np.zeros(reduced_shape) # create a shortened result array
    for idx in np.ndindex(array.shape[0:3]): # for all indices along the first 3 dimensions..
        spiketimes = (array[idx].nonzero()[0] / factor).astype(int) # get spiketimes, divide by factor
        for spiketime in spiketimes: # loop manually, since numpy indexing treats index lists as sets
            spikes_reduced[idx[0],idx[1],idx[2],spiketime] += 1 # increment result array by 1 at the divided spiketimes
    
    if not sum_spikes:
        spikes_reduced  = spikes_reduced >= 1
    spikecount_difference = (np.sum(array) - np.sum(spikes_reduced)).astype(int)
    if spikecount_difference > 0:
        print "Lost "+str(spikecount_difference)+" spikes through resolution reduction."
    return spikes_reduced    

def find_percentiles(y, num_percentiles, return_thresholds=False):
    """ Finds `num_percentiles` equally spaced percentiles of `y` (a monotonically increasing vector),
    i.e. it approximates the inverse `x` of the function `y=f(x)` 
    at `num_percentiles` equally spaced `y`-values between 0 and `y[-1]`.
    
    Arguments:
        y:                  a sequence of monotonically increasing function values
        num_percentiles:    number of percentiles to find between 0 and `y[-1]`.
        return_threshold:   boolean value to indicated whether or not to return the thresholds, too
    Returns:
        percentiles:        `y`-values of the percentiles
        [thresholds:        `x`-values of the percentiles]
    """
    thresholds = np.linspace(0,y[-1],num_percentiles+1)
    percentiles = np.zeros_like(thresholds)
    
    current = 1
    for step in range(T):
        if y[step] > thresholds[current]:
            percentiles[current] = step
            current +=1
    
    percentiles[-1] = len(y)
    if return_thresholds:
        return percentiles, thresholds
    else:
        return percentiles

def calc_cum_abs_deriv(y, sigma=10, order=1):
    """ Calculates the curvature of the spike count vector `y`"""
    return np.cumsum(np.abs(gaussian_filter1d(y, sigma=sigma, order=order)))



# Load the data:
with np.load('data.npz') as archive:
    original_spikes = archive['spikes_outputencoding']

# reduce resolution:
resolution_reduction = 5 # <-- resolution reduction factor
spikes = reduce_resolution(original_spikes,factor=resolution_reduction, sum_spikes=True)

# choose a subset of the data (one cell & one experimental condition):
cell = 1
condition = 1
N = spikes.shape[2] # number of trials
T = spikes.shape[3] # length of a trial
all_spiketrains = spikes[cell,condition,:,:] # N x T array containing only the subset of data that we picked
sum_spiketrains = all_spiketrains.sum(axis=0)
spiketrain0     = all_spiketrains[0]


sigma               = 100/resolution_reduction
spline_order        = 3
num_knots           = 50


cumulative_derivative = calc_cum_abs_deriv(sum_spiketrains, sigma=sigma, order=1)
adaptive_rate_knots   = find_percentiles(cumulative_derivative, num_knots)
knots                 = splines.augknt(adaptive_rate_knots, spline_order)
rate_spline_bases     = np.array(splines.spcol(range(T),knots,spline_order))

## Auto-history in a single trial

We first focus on history effects in a single spike train, then later expand the model back to multiple trials.

As mentioned above, our goal is to add new model features (i.e. new columns in the design matrix) to answer this question: "What change in spike probability at the current step is caused by a spike one (2, 3, 4,...) time steps ago".

A simple way to approach this would be to add one new feature for each length of time we look back. So, we could do the following for each length of time $\delta \in \{1,2,3,4,...,20\}$:
- add an additional column to the design matrix
- in that column, at row (time step) $t$, write a 1 if there was a spike at $t-\delta$.

The rest of the design matrix -- what we call the temporal component -- would remain as in the last session, i.e. just our usual slow-changing basis splines evaluated on $(0,T)$.

If we fit this expanded model to our spike data, we would see for example that the weight at the $\delta = 1$ column comes up as negative. Let's try to explain why. We know that in our data, there is almost never a spike directly at times directly after a spike *did* occur, because of refractoriness. So of course, we would like the model to assign a spiking probability of zero at those time steps. But the model is in a conflict at time steps like that: Our temporal component says that the spike probability is nonzero (since that component changes slowly and so needs to account for all the nonzero spiking activity going on in the general vicinity), but the data at such steps reliably says there is nothing. If the model has no history component (as in the last session), it will solve this by decreasing its weights on the temporal component a little bit. Now, on the other hand, it can solve it by setting a negative weight on the newly introduced $\delta = 1$ column: This results in the desired low probabilities assigned to time steps directly following spikes, all while the slow-changing temporal component can remain as high as it needs to be to explain the surrounding spiking activity. So the interpretation of a negative weight at the  $\delta = 1$ column is: One step after a spike, the spike probability drops down. The other 19 newly added columns would add similar new degrees of freedom, and similar interpretations.

That is the basic idea that carries the rest of this session -- and the same basic idea applies to cross-history. The only problem is that the mentioned scheme is too wasteful: It would require 20 new parameters. It seems implausible that the impact of a spike, say, 17 steps back should be vastly different from the impact of a spike from 19 steps back, so why waste separate parameters on these differences? (Although maybe there *is* a big difference between whether the spike happened 1 step ago or 3 steps ago.)

Hence, instead of adding a separate column for each length of time that we look back, we will add only few columns, filled with increasingly smeared out versions of past spikes -- more smeared out the longer ago the spike took place. 

This will become more concrete after you solved the following exercises.


### Exercise 1a: Auto-history kernels

We want to convolve (filter) our spiketrain with a number of increasingly broad kernels functions. That is, the design matrix column responsible for very recent spikes will result from filtering previous spikes with a kernel situated close to 0 and relatively sharp, while the column responsible for spikes from a further ago will be the result of filtering with a kernel situated further out and broader (Pictures of these things will appear in a minute).

You will now create a set of such kernel functions -- using the familiar tools to create spline basis functions.

* create a logarithmically spaced knot vector. To do this,
    * use np.logspace to create a list of `history_num_knots` numbers between 0 and and $log_{10}($ `history length` $)$ and feed it into the familiar `spline.augknt` function.
    * delete the last knot! this forces the last kernel function to wind down to 0, instead of ending discontinuously.



In [1]:
import splines
import statsmodels.api as sm

history_length = 100/resolution_reduction
history_spline_order = 3                          
history_num_knots = 4

# Create spline history kernel
# history_knots = ...


history_x = np.linspace(1+1e-10,history_length,history_length)                                 
history_spline_bases = splines.spcol(history_x,history_knots,history_spline_order)
num_history_bases  = history_spline_bases.shape[1]


figure()
plot(np.tile(history_knots,(2,1)),np.zeros((2,len(history_knots)))+np.array([[0],[1]]), "k:")
plot(history_x,history_spline_bases);
xlim([0,history_length+1])
xlabel(r"$\delta$")
title("history kernels (colors), knots (dotted)")

figure()
imshow(history_spline_bases,aspect='auto',interpolation='nearest')
ylabel(r"$\delta$")
xlabel("kernel nr.")
title("same thing, different view: history kernels as a 2D plot")

### Exercise 1b: History component of the design matrix

We mentioned that we expand our model's design matrix by additional columns -- not exactly one for each length of time that we look back for previous spikes for, more of an approximation of that. The intuition is the same: We write something in the respective column, at time $t$, whenever there was a spike in the relevant preceding timeframe. You achieve this by filtering (convolving) our own spike train with the history kernels defined above.

* Create the 2D array `history_dm` with dimensions T x `num_history_bases`. 
* In the `i`'th column, it should contain a convolution of the single spiketrain `spiketrain0` with the `i`'th history kernel (that you defined in 1a). Two hints about that:
    * use the `np.convolve` function with `mode='same'`.
    * each kernel has length `history_length`. You need to prepend just as many zeros in front of it (that is, create a "zero-padded" kernel of length 2 \* `history_length`) and apply the convolution only that one.

In [2]:
# history_dm = ...


figure(figsize=(13,3))
plot(spiketrain0[:T/10],'gray')
plot(history_dm[:T/10])
legend(["spikes"]+["basis fct. "+str(i) for i in range(num_history_bases)],fontsize='small')
title("History basis functions (excerpt, one tenth of a trial)")
ylim(-0.1,1.1)

### Exercise 1c: Putting together the design matrix (history + rate)

Stack together the `history_dm` with the old design matrix from last time, which we prepared for you under the name `rate_spline_bases`. The result should be a 2D array `single_trial_dm` with `T` rows and a few more columns than the old design matrix.

If all worked, you will see a rather large plot of the first tenth of the design matrix, now with history component, plotted next to the target spike train. 

**Perhaps with this picture, the explanation given in the beginning begins to make sense:** To estimate the spike probabilities matching our target spiketrain (shown left), a weighted sum of the columns of the design matrix is sought (right). With the 5 new history columns, we can now more easily explain gaps between spikes without needing to touch the slow, temporal component.

In [3]:
# single_trial_dm = ...




assert single_trial_dm.shape == (T,num_history_bases+rate_spline_bases.shape[1])
figure(figsize=(12,15))
ax1 = subplot2grid((1,50),(0, 0),colspan=(1))
imshow(spiketrain0.reshape(-1,1)[:T/5,:], aspect="auto", interpolation="nearest",cmap='gray')
title("target spike train")
xticks([])

ax2 = subplot2grid((1,50),(0, 3),colspan=(48))
imshow(single_trial_dm[:T/5,:], aspect="auto", interpolation="nearest")
yticks([])
ylabel("output dimension (time step)")
xlabel("input dimension (basis function nr)")
title("design matrix (excerpt)")

### Fit model with rate and history component on a single trial

lets do it!

In [5]:
spikes_nospikes = np.array([spiketrain0,resolution_reduction-spiketrain0]).T

model = sm.GLM(spikes_nospikes,single_trial_dm,family=sm.families.Binomial())
fitted_model = model.fit()

prediction = fitted_model.predict()

### Exercise 1d: Analyzing the model

As mentioned above, we can decompose and interpret the different sections of this model separately.

To do this, we select the fitted weights of the different components and apply them to their respective basis functions.

* create the vector `hist_component` by applying the correct section of `fitted_model.params` to `history_spline_bases`. What would be an obvious interpretation of the resulting plot?

* create the vector `rate_component` by applying the correct section of `fitted_model.params` to `rate_spline_bases`. The plot of the temporal component is probably not very impressive -- we'll try again later with more than one trial.


**Hint: ** the `params` have the same order as the columns of your design matrix.

At the bottom, this cell also plots the model's complete output (i.e. the estimated firing probability) for (parts of) the given spiketrain, together with the spikes.


In [4]:
figure()
title('history component of fitted model')
# your code here:
# hist_component = np.dot(history_spline_bases,fitted_model.params[...])


plot(hist_component)
xlim([0,history_spline_bases.shape[0]])



figure(figsize=(15,4))
title('rate component, model influence')
# rate_component = ...

plot(rate_component)
xlim([0,T])




figure(figsize=(15,4))
up = np.max(prediction[10:T/5-10])*1.1
plot(spiketrain0*up,'r',alpha=0.2)
plot(prediction[:T])
xlim([0,T/5])
ylim([0,up])
title('output: firing probability over time (excerpt)')
xlabel('time step')


### Exercise 1d: Adding cross history features

Adding inputs from other cells works very similarly to the auto-history case from exercise 1a, except that now we convolve not with spikes from our own cell, but those from other cells. (In other words, auto-history is the special case.)


* For each of the selected cells whose history we consider (`cross_cells`), create a "sub-designmatrix" `cross_cell_dm` of dimensions `T` x `num_history_bases`. Its `i`'th column should contain the convolution of a spiketrain from the currently selected cell (those spiketrains are provided in `cross_spiketrains0`) with the `i`'th history kernel -- just as in 1a. Use the `np.convolve` function in the same way as before, including the zero padding of the convolution kernel.

The provided code will then stack together these individual per-cell matrices, creating a single `T` x `num_history_bases` \* `len(cross_cells)` array. 

Then, it prepends all of this to our main design matrix.

In [7]:
cross_cells        = [0,2,3,4]
cross_spiketrains0  = spikes[cross_cells,condition,0,:] # the first trial from all cells in cross_cells
cross_cell_dms = []

for c in range(len(cross_cells)):
    cross_cell_dm = np.zeros((T,num_history_bases))
    for i in range(num_history_bases):
#        padded_kernel = ...
#        cross_cell_dm[...] = ...

    cross_cell_dms.append(cross_cell_dm)
    
# stack up the history basis functions for different cross cells
cross_history_dm = np.hstack(cross_cell_dms)

# stack it all up
single_trial_dm = np.hstack([history_dm, cross_history_dm, rate_spline_bases])

figure(figsize=(12,32))
imshow(single_trial_dm, aspect="auto", interpolation="nearest")
title("design matrix with cross history, auto history and rate component (complete single trial)")

### Fitting all of it (single trial)
Let's do it again!

In [8]:
model = sm.GLM(spikes_nospikes,single_trial_dm,family=sm.families.Binomial())
fitted_model = model.fit()
prediction = fitted_model.predict()

###  Analyzing the model (single spike train)

As mentioned numerous times, we can decompose and interpret the different sections of this model separately. To do this, we select the fitted weights of the different components and apply them to their respective basis functions.

In [9]:
auto_history_kernel = np.dot(history_spline_bases,fitted_model.params[:num_history_bases])
cross_history_kernels = np.hstack([np.dot(history_spline_bases,fitted_model.params[(i+1)*num_history_bases:(i+2)*num_history_bases]).reshape((-1,1)) for i in range(4)])

auto_history_component = np.dot(single_trial_dm[:,:num_history_bases],fitted_model.params[:num_history_bases])
cross_history_components = np.dot(single_trial_dm[:,num_history_bases:(len(cross_cells)+1)*num_history_bases],fitted_model.params[num_history_bases:(len(cross_cells)+1)*num_history_bases]).reshape((-1,len(cross_cells)))
rate_component =np.dot(single_trial_dm[:,(len(cross_cells)+1)*num_history_bases:],fitted_model.params[(len(cross_cells)+1)*num_history_bases:])



figure()
plot(auto_history_kernel)
xlim([0,len(auto_history_kernel)])
title("auto history combined kernel ")

figure()
plot(cross_history_kernels)
xlim([0,len(auto_history_kernel)])
title("cross history combined kernels (different cells)")

figure(figsize=(15,4))
plot(auto_history_component[:T/10])
xlim([0,T/10])
title("auto history, model influence over time (excerpt)")

figure(figsize=(15,4))
plot(cross_history_components[:T/10])
xlim([0,T/10])
title("cross history, model influence over time (excerpt), different cells")


figure(figsize=(15,4))
plot(rate_component[:T])
xlim([0,T])
title("rate component, influence over time")


figure(figsize=(15,4))
up = np.max(prediction[10:T/5-10])*1.1
plot(spiketrain0*up,'r',alpha=0.2)
plot(prediction[:T])
xlim([0,T/5])
ylim([0,up])
title('firing probability over time (excerpt)')
xlabel('time step')


## Multiple trials

That is one sad looking rate component. 

So far, we fitted the model on only a single spiketrain. Now, the remaining N-1 trials come into play, which should result in a much more interesting model fit.

But incorporating multiple trials poses a bit of a problem. In the previous session, we could simply add up the spike counts from different trials, which has lead to a target vector of length `T` and a design matrix with `T` rows. Not so here: Our model now has basis functions that depend not just on the step number, but on the timing of individual spikes within the current trial. If we add up all spike counts from different trials, we lose this information.

The trials are, however, independent. We can therefore chain them together into a single, large "trial" of length T\*N, which also leads to a design matrix of that many rows. We can construct the history sections of this very long design matrix as shown before, and only need to make sure that the basis functions of our temporal component repeat always at the right times (namely, every `T` steps).

### Exercise 2a: Design matrix for multiple trials

You will now create the design matrix for the full, multi-trial model. 


* Create a 2D array called `dm` with T\*N rows, structured as follows (so that later plot code works):
    * the first `num_history_bases` columns  should be the auto history columns
    * next should be 4 \* `num_history_bases` cross history columns
    * the remaining columns are the temporal / rate component. It repeats every `T` rows.
  
**Hint 1:** Save some work: auto-history is just cross-history with the cell `cell`, i.e. our own one.

**Hint 2:** To chain together spiketrains of one cell & condition across all trials , you can use `allspikes = spikes[cell,condition,:,:].ravel()`. This returns a 1D array / vector of length T\*N.

**Hint 3:** check out `np.hstack` and `np.tile`

In [8]:
# your code here



assert len(cross_cells) == 4 
assert dm.shape == (T*N,5*num_history_bases+rate_spline_bases.shape[1])

figure(figsize=(12,32))
imshow(dm, aspect="auto", interpolation="nearest")
title('design matrix (all trials)')
xlabel('basis function nr')
ylabel('step*trial')

## Fit all the things

What has changed in the behaviour of the different components, relative to the single trial fit?

In [11]:
spikes_nospikes = np.array([all_spiketrains.ravel(),resolution_reduction-all_spiketrains.ravel()]).T
model = sm.GLM(spikes_nospikes,dm,family=sm.families.Binomial())
fitted_model = model.fit()
prediction = fitted_model.predict()

In [10]:
auto_history_kernel = np.dot(history_spline_bases,fitted_model.params[:num_history_bases])
cross_history_kernels = np.hstack([np.dot(history_spline_bases,fitted_model.params[(i+1)*num_history_bases:(i+2)*num_history_bases]).reshape((-1,1)) for i in range(4)])
auto_history_component = np.dot(single_trial_dm[:,:num_history_bases],fitted_model.params[:num_history_bases])
cross_history_components = np.dot(single_trial_dm[:,num_history_bases:(len(cross_cells)+1)*num_history_bases],fitted_model.params[num_history_bases:(len(cross_cells)+1)*num_history_bases]).reshape((-1,len(cross_cells)))
rate_component =np.dot(single_trial_dm[:,(len(cross_cells)+1)*num_history_bases:],fitted_model.params[(len(cross_cells)+1)*num_history_bases:])

figure()
plot(auto_history_kernel)
xlim([0,len(auto_history_kernel)])
title("auto history combined kernel ")

figure()
plot(cross_history_kernels)
xlim([0,len(auto_history_kernel)])
title("cross history combined kernels, different cells")

figure(figsize=(15,4))
plot(auto_history_component[:T/10])
xlim([0,T/10])
title("auto history, model influence over time (excerpt)")

figure(figsize=(15,4))
plot(cross_history_components[:T/10])
xlim([0,T/10])
title("cross history, model influence over time (excerpt), different cells")


figure(figsize=(15,4))
plot(rate_component[:T])
xlim([0,T])
title("rate component, model influence over time")
[t.set_color('red') for t in gca().xaxis.get_ticklabels()]

figure(figsize=(15,4))
up = np.max(prediction[10:T/5-10])*1.1
plot(spiketrain0*up,'r',alpha=0.2)
plot(prediction[:T])
xlim([0,T/5])
ylim([0,up])
title('output: firing probability over time (excerpt)')
xlabel('time step')

Compared to our single-trial fit that showed clear signs of overfitting, our rate component has improved considerably!
This results in a higher likelihood of the spike-train, as can be seen by the fact that spikes occur at positions, where the spiking probability is higher.

## Generating data
So far, we constructed a probabilistic model that allows us to calculate the likelihood of observed spike-trains.
[But wait, there is more!](http://blogs.edweek.org/edweek/the_startup_blog/assets_c/2014/02/a78ae76da19c1a0f9e0e9b2f7e6229e70bd36cf7bc5b2f29b5f8900face50234%5B1%5D-thumb-autox384-6965.jpg)
Since we model the whole posterior distribution (a binomial distribution of spikes over time), we might want to create surrogate data from our model; either to use as data for other purposes, or for comparison to the data recorded from real neurons. This might help us to see systematic differences between the real data and our model predictions and thus highlight the deficiencies of our model.

### GLM Review (for the curious)
The model fitting procedure for the GLM described above assumes that the number of spikes $y$ generated by the neuron in time bin $t$ is a binomial random variable with some firing probability $\mu(t)$. Since the binomial distribution is a member of an exponential family, its *natural parameter*, $\eta(t)$, can be expressed as a function (the so called *link-function*) of the expected value of its *sufficient statistic*. In the case of a binomial, this means that $\eta(t) = \text{logit}(\mathbb{E}[y(t)]) = \text{logit}(\mu(t)) = \log(\frac{\mu(t)}{1-\mu(t)})$ (for a binomial random variable $y$, the sufficient statistic is just $y$ and the canonical link function is the $\text{logit}$ function).
Writing this another way gives us the expected value of $y$ as a function of $\eta$: $\mathbb{E}[y(t)] = \mu(t) = \text{logit}^{-1}(\eta(t)) = \frac{1}{1+\exp(-\eta(t))}$. 

We now assume that $\eta := X \beta$ is a linear combination of the basis functions in the columns of our design matrix $X$ with the weights $\beta$. The GLM-fitting procedure now tries to find optimal weights $\hat \beta$ for which the vector $\hat \mu =  \frac{1}{1+\exp(-X \hat \beta)}$ has the maximal likelihood.
We can use the same expression to calculate a prediction of our model, given a design matrix $X$ and a parameter vector $\beta$.

### Exercise 3a
We now want to generate data using our GLM fitted above.
At first, we are not interested in the auto-history component that the model learned and will thus set its parameters to zero.

Your job is to use the model to sample $N=20$ spike-trains of surrogate data.
Each artificial spike train should use the part of the design matrix for the corresponding trial (because e.g. the cross-history components are different for different trials). For this, you need to do the following:

 * calculate the vector $\mu$ of firing probabilities over time (where $\mu =  \frac{1}{1+\exp(-X \hat \beta)}$)
     * In the formula, $X$ is the design matrix used in the fitting procedure and ...
     * ... $\hat \beta$ is the fitted parameter-vector `beta` (with the weights for the auto-history component zeroed out).
 * use the predicted firing probabilities $\mu$ to draw $20$ spike trains $y$ of length $T$, sampled from a binomial random variable with means $\mu$
     * **Hint 1:** the parameter $n$ of the binomial distribution is `resolution_reduction`
     * **Hint 2:** use the imported function `binomial_rv` to draw from a random binomial variable; it can draw from a whole vector of random variables at once if you supply it with a vector of probabilities.
 * store the artificial spike trains in the $20\times T$ array `generated_spiketrains`
     * **Hint 3:** If you want the shortest solution: It's possible to generate just one spike train of length $N\cdot T$ using the whole design matrix and then use `np.reshape` to turn it back into the desired $N\times T$ array `generated_spiketrains`.

In [11]:
from numpy.random import binomial as binomial_rv

beta = fitted_model.params.copy()
beta[:num_history_bases] = 0

generated_spiketrains = np.zeros((N,T))

#....


# Code for plotting surrogate and real spike-trains:
spike_y, spike_t = np.where(generated_spiketrains)
red_dots=plot(spike_t,N-spike_y,"r|",alpha=0.7,ms=7)
spike_y, spike_t = np.where(all_spiketrains)
black_dots = plot(spike_t,40-spike_y,"k|",alpha=0.5,ms=7)
title("Real data vs. surrogate data w/o auto-history")
plot([0,T],[20.5,20.5],"k-")
yticks(np.arange(0,41,5))
gca().set_yticklabels(np.hstack([np.arange(0,21,5),np.arange(5,21,5)]))
xlim([0,T])
legend([black_dots[0], red_dots[0]], ["trial spike trains","generated spike trains"], bbox_to_anchor=(0., -0.2, 1., -0.2), loc=3, ncol=2, mode="expand", borderaxespad=0)

We can see that the rate and cross-history components of our model can capture regions where the neuron spikes a lot and regions where it doesn't. But without the learned auto-history component our model fails to account for the fact that even in regions of high firing probabilities the real neuron does not fire spikes in very rapid succession.

To generate more realistic spike trains, we want to include the auto-history component, as well.
This introduces a bit of a complication, since the auto-history, as the name implies, depends on the neuron's **own** spiking history. The spike generation process at each time step thus depends on the previously generated spikes, as well as on the cross-history and rate component.

### Exercise 3b:

We now want to replace the $N\times T$ array `generated_spiketrains` from exercise 3a by an array (same shape) of spikes drawn from a model that uses the learnt auto-history component, as well. To this end, you need to do the following:

 * Iterate over all `N` trials and all time `T` steps and do the following for each:
     * calculate the linear predictor `eta`:
         * the combined effect of the cross-history and the rate component for each trial and time step can be read from `eta_r_c`
         * the effect of the auto-history must be calculated from the neuron's spiking history in the same trial over an interval of length `history_length` right before the current time step and added to the effect of cross-history and rate
         * **Hint 1:** you can use the dot-product of the neuron's recent history and the `auto_history_kernel` to calculate the effect of the neuron's auto-history
         * **Hint 2:** you can start generating spikes after `history_length` time steps; this might make your code simpler
         * **Hint 3:** to extract the last `l` elements before index `k` from an array `a`, you can write `a[k-l:k]`
     * use `eta` to calculate the firing probability `mu`
     * draw a random number of spikes from a binomial distribution with mean `mu`
     * store the number of spikes into `generated_spiketrains[trial,step]`

In [12]:
eta_r_c = np.dot(dm, beta).reshape((N,T))
generated_spiketrains = np.zeros((N,T))

# ...

# Code for plotting surrogate and real spike-trains:
spike_y, spike_t = np.where(generated_spiketrains)
red_dots=plot(spike_t,N-spike_y,"r|",alpha=0.7,ms=7)
spike_y, spike_t = np.where(all_spiketrains)
black_dots = plot(spike_t,40-spike_y,"k|",alpha=0.5,ms=7)
title("Real data vs. surrogate data w/ auto-history")
plot([0,T],[20.5,20.5],"k-")
yticks(np.arange(0,41,5))
gca().set_yticklabels(np.hstack([np.arange(0,21,5),np.arange(5,21,5)]))
xlim([0,T])
legend([black_dots[0], red_dots[0]], ["trial spike trains","generated spike trains"], bbox_to_anchor=(0., -0.2, 1., -0.2), loc=3, ncol=2, mode="expand", borderaxespad=0)

We observe that including the auto-history component has resulted in much sharper peaks in the regions of high firing probability, leading to much more realistic artificial spike trains.

### Summary
In this part, we saw how to introduce the dependency of a stochastic process on its own history (auto-correlation / auto-history) as well as on filtered external variables (cross-correlation / cross-history) in addition to the time dependent component we used in part 2.

Since GLMs provide not just an expected value of a prediction (a regression line), but the whole posterior distribution, we used our model to generate surrogate data that fits our measured data quite well.

That's it for Part 3!
==========================